In [36]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import time

def login_and_scrape_data(fullname, username, password):
    chromedriver_path = "chromedriver.exe"
    
    # Use the ChromeOptions to set the executable_path
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument(f"executable_path={chromedriver_path}")
    
    with webdriver.Chrome(options=chrome_options) as driver:
        driver.get('https://phongdaotao2.ntt.edu.vn/sinh-vien-dang-nhap.html')
        time.sleep(5)
    # Find necessary elements
        find_account_input = driver.find_element(By.ID, "UserName")
        find_password_input = driver.find_element(By.ID, "Password")
        find_captcha_input = driver.find_element(By.ID, "Captcha")
        signin_button = driver.find_element(By.CLASS_NAME, "form-button")
 
    # Enter username and password
        find_account_input.clear()
        find_password_input.clear()

        find_account_input.send_keys(username)
        find_password_input.send_keys(password)

        find_captcha_input.clear()
        captcha_input = input('Nhập captcha: ').strip()
        find_captcha_input.send_keys(captcha_input)
        signin_button.click()

    # Redirect to attendance page and click AccordingAll button
        driver.get('https://phongdaotao2.ntt.edu.vn/chuong-trinh-khung.html')
        find_AccordionAll_button = driver.find_element(By.ID, "OpenAccordionAll")
        find_AccordionAll_button.click()


    # Get column names
        find_heading_element = driver.find_element(By.XPATH, '//*[@id="contnet"]/div/div[2]/div/div/div[2]/div/table/thead')
        heading_elements = find_heading_element.find_elements(By.TAG_NAME, 'th')
    # Get data
        tr_values = []
        tbodys = driver.find_elements(By.TAG_NAME, 'tbody')[:-1]
        for tbody in tbodys:
            trs = tbody.find_elements(By.TAG_NAME, 'tr')
            for tr in trs[2:]:  # Bỏ qua tr[1] và tr[2]
                tds = tr.find_elements(By.TAG_NAME, 'td')
                row_data = [td.text for td in tds]
                tr_values.append(row_data)
        df = pd.DataFrame(tr_values)
        return df

result_df1 = login_and_scrape_data('Cao Đăng Khoa','2000001001','thuytrang14092002')
result_df1


The chromedriver version (124.0.6367.201) detected in PATH at f:\Learning\FinalProject\notebooks\crawl\chromedriver.exe might not be compatible with the detected chrome version (125.0.6422.78); currently, chromedriver 125.0.6422.78 is recommended for chrome 125.*, so it is advised to delete the driver in PATH and retry


,0,1,2,3,4,5,6,7,8,9,...,610,611,612,613,614,615,616,617,618,619
0,1,Kỹ năng giao tiếp,1001070006,,2,30,0,0,,,...,None,None,None,None,None,None,None,None,None,None
1,2,Dịch vụ Hành chính,1001074618,,3,45,0,0,,,...,None,None,None,None,None,None,None,None,None,None
2,3,Công nghệ thông tin,1001074673,,3,0,90,0,,,...,None,None,None,None,None,None,None,None,None,None
3,4,Tiếng Anh 1,1001074730,,6,60,60,0,,,...,None,None,None,None,None,None,None,None,None,None
4,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
223,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
224,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
225,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None


In [37]:
result_df1

,0,1,2,3,4,5,6,7,8,9,...,610,611,612,613,614,615,616,617,618,619
0,1,Kỹ năng giao tiếp,1001070006,,2,30,0,0,,,...,None,None,None,None,None,None,None,None,None,None
1,2,Dịch vụ Hành chính,1001074618,,3,45,0,0,,,...,None,None,None,None,None,None,None,None,None,None
2,3,Công nghệ thông tin,1001074673,,3,0,90,0,,,...,None,None,None,None,None,None,None,None,None,None
3,4,Tiếng Anh 1,1001074730,,6,60,60,0,,,...,None,None,None,None,None,None,None,None,None,None
4,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
223,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
224,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None
225,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,None


In [38]:
result = result_df1.iloc[:64, 1:7]
# Xóa các dòng có giá trị None
result_none = result[result.isna().all(axis=1)].index
result = result.drop(result_none)
result = result.drop(56, axis = 0)
result.columns = ['course name', 'course id', 'course', 'credit', 'Theoretical session', 'Lab session']
result['total sessions'] = result['Theoretical session'].astype('int') + result['Lab session'].astype('int')
with pd.ExcelWriter('curriculum_framework.xlsx', engine='xlsxwriter') as writer:
    result.to_excel(writer, sheet_name='Data', index=False)

In [39]:
result.head(4)

,course name,course id,course,credit,Theoretical session,Lab session,total sessions
0,Kỹ năng giao tiếp,1001070006,,2,30,0,30
1,Dịch vụ Hành chính,1001074618,,3,45,0,45
2,Công nghệ thông tin,1001074673,,3,0,90,90
3,Tiếng Anh 1,1001074730,,6,60,60,120
